In [62]:
import os
import email
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

from sklearn.cluster import KMeans

In [2]:
types = Counter()
msgs = []
trigger = True
for root,dirs,files in os.walk('/Users/taddbackus/School/fall23/qtw/cs3/SpamAssassinMessages/'):
    for f in files:
        with open(os.path.join(root,f),'r',encoding='latin-1') as file_handle:
            msg = email.message_from_file(file_handle,)
            type_ = msg.get_content_type()
            types[type_]+=1
            if type_ == 'multipart/mixed' and trigger:
                print(root,f)
                print('______________')
                trigger = False
                SAMPLE = msg.get_payload()
print(types)


/Users/taddbackus/School/fall23/qtw/cs3/SpamAssassinMessages/spam 0343.0630afbe4ee1ffd0db0ffb81c6de98de
______________
Counter({'text/plain': 7414, 'text/html': 1193, 'multipart/alternative': 326, 'multipart/signed': 180, 'multipart/mixed': 179, 'multipart/related': 56, 'multipart/report': 5, 'text/plain charset=us-ascii': 1})


In [23]:
def return_txt(msg):
    if msg.get_content_type() == 'text/plain':
        return msg.get_payload()
    if msg.get_content_type() == 'text/html':
        return "***Austin is better at beautiful soup than me"
    else:
        for item in msg.get_payload():
            return return_txt(item)
            break

In [45]:
file_names = []
label = []
messages = []
msgType = []
htmlMsgs = []
msgToFix = []
for root,dirs,files in os.walk('/Users/taddbackus/School/fall23/qtw/cs3/SpamAssassinMessages/'):
    for f in files:
        with open(os.path.join(root,f),'r',encoding='latin-1') as file_handle:
            msg = email.message_from_file(file_handle)
            try:
                text = return_txt(msg)
                if '***Austin' in text:
                    htmlMsgs.append([text,root,f])
                else:
                    messages.append(return_txt(msg))
                    if 'spam' in root:
                        label.append(1)
                    else:
                        label.append(0)
            except:
                msgToFix.append(os.path.join(root,f))
                print(f"{root,f} failed to parse")

('/Users/taddbackus/School/fall23/qtw/cs3/SpamAssassinMessages/spam', '00467.5b733c506b7165424a0d4a298e67970f') failed to parse
('/Users/taddbackus/School/fall23/qtw/cs3/SpamAssassinMessages/spam', '0478.16f274ffd3e884a574c9461012cadea6') failed to parse
('/Users/taddbackus/School/fall23/qtw/cs3/SpamAssassinMessages/spam_2', '00204.4cf15f97b8ea08bfafab7d5091b8fbe7') failed to parse
('/Users/taddbackus/School/fall23/qtw/cs3/SpamAssassinMessages/spam_2', '01214.973b4598b630a989967ff69b19f95d4a') failed to parse


In [63]:
for file in msgToFix:
    with open(file,'r',encoding='latin-1') as badFile:
        msg = email.message_from_file(badFile)
        for m in msg:
            print(m[2])

t
l
c
c
c
s
o


IndexError: string index out of range

In [33]:
print(len(messages))
print(len(label))
print(len(htmlMsgs))

8044
8044
1306


In [36]:
vectorizer = CountVectorizer()
textVector = vectorizer.fit_transform(messages)

In [64]:
kMeans = KMeans(random_state=42)
kMeans.fit(textVector)

0.0002486325211337643


/Users/taddbackus/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [69]:
textData = textVector.toarray()
textData.shape

(8044, 76776)

In [70]:
cluster_column = np.array(kMeans.labels_).reshape(-1, 1)
textClusters = np.hstack((textData,cluster_column))
textClusters.shape

(8044, 76777)

In [41]:
nb = MultinomialNB()

In [71]:
nb.fit(textClusters,label)

MultinomialNB()

In [72]:
cross_val_score(nb,textClusters,label,cv=5,n_jobs=-1,scoring='accuracy')

array([0.96146675, 0.99129894, 0.98881293, 0.98632691, 0.98818408])